In [38]:
import random

## Vacuum Reflex Agent in 4 Room Grid

In [39]:
class VacuumAgent:
    def __init__(self):
        self.loc = (0, 0)
        self.dr = {
            (0, 0): {(0, 1): "down", (1, 0): "right"},
            (1, 0): {(0, 0): "left", (1, 1): "down"},
            (0, 1): {(0, 0): "up", (1, 1): "right"},
            (1, 1): {(1, 0): "up", (0, 1): "left"}
        }
    def act(self, percept):
        ix, iy, status = percept 
        if status == 'Dirty':
            return 'Suck'
        else:
            new_loc = random.choice(list(self.dr[self.loc].keys()))
            move = self.dr[self.loc][new_loc]
            self.location = new_loc
            return f'Current place already clean, moving {move}. New location is {new_loc}'

In [40]:
# running the simulation for 10 random steps
rooms_cleaned = 0
movements = 0
n = 10
agent = VacuumAgent()
for i in range(n):
    x = random.choice([0, 1])
    y = random.choice([0, 1])
    status = random.choice(['Clean', 'Dirty'])
    result = agent.act((x, y, status))
    print('========')
    print(f'Percept ({x}, {y}, {status})')
    print(result)
    print('========')
    if result == 'Suck':
        rooms_cleaned += 1
    else:
        movements += 1
print(f'After {n} steps, rooms cleaned = {rooms_cleaned}, movements = {movements}')

Percept (1, 1, Dirty)
Suck
Percept (1, 0, Clean)
Current place already clean, moving right. New location is (1, 0)
Percept (0, 0, Clean)
Current place already clean, moving right. New location is (1, 0)
Percept (1, 1, Dirty)
Suck
Percept (1, 1, Dirty)
Suck
Percept (0, 1, Clean)
Current place already clean, moving down. New location is (0, 1)
Percept (1, 0, Dirty)
Suck
Percept (0, 0, Clean)
Current place already clean, moving right. New location is (1, 0)
Percept (1, 0, Dirty)
Suck
Percept (1, 1, Clean)
Current place already clean, moving right. New location is (1, 0)
After 10 steps, rooms cleaned = 5, movements = 5


## Model Based Vacuum Agent

In [41]:
class ModelVacuumAgent:
    def __init__(self):
        self.loc = (0, 0)
        self.dr = {
            (0, 0): {(0, 1): "down", (1, 0): "right"},
            (1, 0): {(0, 0): "left", (1, 1): "down"},
            (0, 1): {(0, 0): "up", (1, 1): "right"},
            (1, 1): {(1, 0): "up", (0, 1): "left"}
        }
        # initializing some rooms to clean and some to dirty
        self.room_status = {
            (0, 0): "Clean",
            (1, 0): "Dirty",
            (0, 1): "Clean",
            (1, 1): "Dirty"
        }
    def act(self, percept):
        ix, iy, status = percept
        all_cleaned = True
        for i in list(self.room_status.keys()):
            if self.room_status[i] == 'Dirty':
                all_cleaned = False 
                break
        if all_cleaned:
            return 'Terminate'
        elif status == 'Dirty':
            self.room_status[(ix, iy)] = 'Clean'
            return 'Suck'
        else:
            new_loc = random.choice(list(self.dr[self.loc].keys()))
            move = self.dr[self.loc][new_loc]
            self.location = new_loc
            return f'Current place already clean, moving {move}. New location is {new_loc}'
            

In [46]:
# running the simulation for 10 random steps
rooms_cleaned_model = 0
movements_model = 0
m = 10
model_agent = ModelVacuumAgent()
for i in range(m):
    x = random.choice([0, 1])
    y = random.choice([0, 1])
    status = random.choice(['Clean', 'Dirty'])
    result = model_agent.act((x, y, status))
    if result == 'Terminate':
        print("*** All rooms are clean, agent terminated***")
        break
    print('========')
    print(f'Percept ({x}, {y}, {status})')
    print(result)
    print('========')
    if result == 'Suck':
        rooms_cleaned += 1
    else:
        movements += 1
print(f'After {n} steps, rooms cleaned = {rooms_cleaned_model}, movements = {movements_model}')

Percept (1, 0, Clean)
Current place already clean, moving right. New location is (1, 0)
Percept (1, 1, Dirty)
Suck
Percept (1, 1, Clean)
Current place already clean, moving down. New location is (0, 1)
Percept (0, 1, Dirty)
Suck
Percept (1, 1, Clean)
Current place already clean, moving right. New location is (1, 0)
Percept (1, 1, Dirty)
Suck
Percept (0, 1, Clean)
Current place already clean, moving down. New location is (0, 1)
Percept (1, 0, Dirty)
Suck
*** All rooms are clean, agent terminated***
After 10 steps, rooms cleaned = 0, movements = 0


Why does the simple reflex agent perform redundant actions in the grid environment? <br>
Simple reflex agents only use current percepts without memory. In a grid, a clean vacuum at randomly moves right/down. If it returns to later (clean), it moves again—endlessly looping between clean cells since it forgets past visits.

How does the internal model improve agent performance? <br>
The model tracks visited locations and dirt status history. Instead of random movement, it systematically explores unvisited cells or avoids recleaning known-clean areas, eliminating redundant loops.

Compare the simple reflex and model-based agents in terms of:
a. Rationality
b. Efficiency
c. Scalability <br>
Simple reflex agents have lower rationality because they repeat actions on already-clean cells without memory, while model-based agents make more rational decisions by tracking world state to avoid redundancy. Efficiency is poor for simple reflex agents due to endless loops between clean cells, whereas model-based agents explore systematically without repeats. For scalability, simple reflex fails completely in larger grids as looping worsens exponentially, but model-based agents scale better by maintaining an internal world model regardless of grid size.

Would the model-based agent still work correctly in a partially observable environment? Justify. <br>
Yes, it works better. The internal model infers hidden state from partial percepts + history. For example, if dirt was sucked at previously, the model assumes it stays clean even if current percept doesn't show location. Simple reflex would re-suck unnecessarily.